In [ ]:
import requests
import urllib
import json
import csv
import pandas as pd
import configparser
import json
from pandas.io.json import json_normalize
import os
import glob

In [ ]:
folder = 'PATH_TO_REPO/Review4Repair_mapped_code_with_comments/'
project_folders = [name for name in os.listdir(folder) if os.path.isdir(os.path.join(folder, name))]

In [ ]:
code_comment = pd.DataFrame()
for proj in project_folders: 
    try:
        with open('PATH_TO_REPO/Review4Repair_mapped_code_with_comments/' + proj +
                  '/gerrit_db_' + proj + '.json', encoding="utf8") as f:
            proj_gerrit = pd.DataFrame(json.load(f))
        with open('PATH_TO_REPO/Review4Repair_mapped_code_with_comments/' + proj +
                  '/Downloaded_Codes_'+ proj +'/' + proj + '_MapJSON.json', encoding="utf8") as f: 
            raw_data = json.loads(f.read())
            code_comment = extract_lines_from_patches(proj, proj_gerrit, raw_data)
            print(code_comment)
    except:
        print('dir not found')

In [ ]:
def getFileExtenion(filepath):
    path_word_list = list(filepath.split("."))
    length = len(path_word_list)
    return path_word_list[length-1]

In [ ]:
def extract_lines_from_patches (proj, proj_gerrit, file_mapper):
    lines = []
    messages = []
    files =  []
    df_lines_messages = pd.DataFrame(columns=['project', 'Id', 'revision', 'content', 'review_comment', 'filename', 'commented_lines' ])

    for key1, row in proj_gerrit.iteritems():
        try:
                for key2, elem in file_mapper.iteritems():
                    if elem["encoded_file"].lower() == row["file_name"].lower():
                        file_extension = getFileExtenion(elem["Java_file"])
                        directory = str(elem["encoded_file"])
                        index_reviewed_line = row["line_number"]
                        base_patch = str(row["base_patch_number"]) #the physical file name, corresponding to a patch number
                        
                        file = open('PATH_TO_REPO/'+ proj + '/Downloaded_Codes_' +proj + '/' + directory + '/'+ 
                                    base_patch + '.'+ file_extension) 

                        content = file.readlines()
                        lines.append(content[index_reviewed_line])
                        messages.append(row['message'])
                        files.append(elem["Java_file"].lower())   
                        print('found')
                        break

                print('iteration ' + str(key1) + ' out of: ' + str(len(proj_gerrit)))

                df_code_comments = pd.DataFrame({'project': 'eclipse',
                                                              'Id': 'not given',
                                                              'revision': 'not given',
                                                              'content': lines,
                                                              'review_comment': messages,
                                                              'filename': files})
                df_code_comments.to_csv('PATH_TO_REPO' + proj + '_comments_code_raw.csv', index=False, sep='$')
                return df_code_comments
        except (Exception):
                print(Exception)